In [ ]:

import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq
from dataclasses import dataclass
import hvplot.pandas 
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# hv.renderer('bokeh').theme = 'dark_minimal'

In [ ]:
# dataset_root = Path(r"C:\Users\Raffael\Documents\Datasets\alpiq_2023") # Raw string works without escaping \
# dataset_root = Path(r"C:/Users/jadbh\Documents/Swisse/EPFL/courses/Fall 2024/Machine Learning for Predictive Maintenance/project/Dataset")
# dataset_root = Path(r"C:\Users\jadbh\Documents\Swisse\EPFL\courses\Fall 2024\Machine Learning for Predictive Maintenance\project\team repo\Machine-Learning-for-Predictive-Maintenance-project\Dataset")
dataset_root = Path(r'Dataset')

@dataclass
class Case():
    info: pd.DataFrame
    measurements: pd.DataFrame


class RawDataset():

    def __init__(self, root, unit = "VG4", load_training=False, load_synthetic=False) -> None:
        
        
        read_pq_file = lambda f: pq.read_table(root / f).to_pandas()
        
        
        cases = {
            "test": [f"{unit}_generator_data_testing_real_measurements.parquet", root / f"{unit}_generator_data_testing_real_info.csv" ], 
        }
        
        if load_training:
            cases = {
                **cases,
                "train": [f"{unit}_generator_data_training_measurements.parquet", root / f"{unit}_generator_data_training_info.csv" ], 
            }
        
        if load_synthetic:
            cases = {
                **cases,
                "test_s01": [f"{unit}_generator_data_testing_synthetic_01_measurements.parquet", root / f"{unit}_generator_data_testing_synthetic_01_info.csv"], 
                "test_s02": [f"{unit}_generator_data_testing_synthetic_02_measurements.parquet", root / f"{unit}_generator_data_testing_synthetic_02_info.csv"]
            }
        
        
        self.data_dict = dict()
        
        for id_c, c in cases.items():
            # if you need to verify the parquet header:
            # pq_rows = RawDataset.read_parquet_schema_df(root / c[0])
            info = pd.read_csv(c[1])
            measurements = read_pq_file(c[0])
            self.data_dict[id_c] = Case(info, measurements)
            
        
        
    @staticmethod
    def read_parquet_schema_df(uri: str) -> pd.DataFrame:
        """Return a Pandas dataframe corresponding to the schema of a local URI of a parquet file.

        The returned dataframe has the columns: column, pa_dtype
        """
        # Ref: https://stackoverflow.com/a/64288036/
        schema = pq.read_schema(uri, memory_map=True)
        schema = pd.DataFrame(({"column": name, "pa_dtype": str(pa_dtype)} for name, pa_dtype in zip(schema.names, schema.types)))
        schema = schema.reindex(columns=["column", "pa_dtype"], fill_value=pd.NA)  # Ensures columns in case the parquet file has an empty dataframe.
        return schema
    

rds_u4 = RawDataset(dataset_root, "VG4", load_synthetic=False, load_training=True)
rds_u5 = RawDataset(dataset_root, "VG5", load_synthetic=True, load_training=True)
rds_u6 = RawDataset(dataset_root, "VG6", load_synthetic=True)

In [ ]:
vg5_train_meas = rds_u5.data_dict["train"].measurements
vg5_train_info = rds_u5.data_dict["train"].info
vg5_test_meas = rds_u5.data_dict["test"].measurements
vg5_test_info = rds_u5.data_dict["test"].info

In [ ]:
vg5_train_filt_turbine = vg5_train_meas [ (vg5_train_meas['equilibrium_turbine_mode'] == True)] 
                                        #    ((vg5_train_meas['equilibrium_pump_mode'] == True) & (vg5_train_meas['short_circuit_mode'] == False)) |
                                        #    ((vg5_train_meas['equilibrium_pump_mode'] == True) & (vg5_train_meas['short_circuit_mode'] == True) & (vg5_train_meas['equilibrium_short_circuit_mode'] == True)) ]

In [ ]:
vg5_s1 = rds_u5.data_dict["test_s01"].measurements          # synthetic testing
vg5_s1_filt_turbine = vg5_s1 [ (vg5_s1['equilibrium_turbine_mode'] == True) ]        # filtered synthetic testing
                    # ((vg5_s1['equilibrium_pump_mode'] == True) & (vg5_s1['short_circuit_mode'] == False)) |
                    # ((vg5_s1['equilibrium_pump_mode'] == True) & (vg5_s1['short_circuit_mode'] == True) & (vg5_s1['equilibrium_short_circuit_mode'] == True)) ]

In [ ]:
vg5_train_meas.reset_index(inplace=True)
vg5_train_filt_turbine = vg5_train_meas [ (vg5_train_meas['equilibrium_turbine_mode'] == True) ]
                                        #    ((vg5_train_meas['equilibrium_pump_mode'] == True) & (vg5_train_meas['short_circuit_mode'] == False)) |
                                        #    ((vg5_train_meas['equilibrium_pump_mode'] == True) & (vg5_train_meas['short_circuit_mode'] == True) & (vg5_train_meas['equilibrium_short_circuit_mode'] == True)) ]

In [ ]:
# # summary of VG5 useful variables

# vg5_train_info
# vg5_train_filt      # equilibirum
# vg5_train_meas

# vg5_test_info
# vg5_test_meas

# vg5_s1
# vg5_s1_filt         # equilibrium

# print()

In [ ]:
df = vg5_train_filt_turbine
print(df.columns)
df.head(3)

In [ ]:
print("OG columns: ", df.columns.to_list())
print("OG columns length: ", len(df.columns.to_list()))
df.drop(columns=df.loc[:, 'machine_on':'equilibrium_short_circuit_mode'].columns, inplace=True)
df.drop(['index'], axis = 1, inplace=True)
print("final columns:       ", df.columns.to_list())
print("final columns lengthL", len(df.columns.to_list()))
df.sample(3)

In [ ]:
class SlidingWindowDataset(Dataset):
    def __init__(self, dataframe, feature_columns, window_size, step_size=1, max_gap=10, stride = 2):
        """
        Args:
            dataframe (pd.DataFrame): The dataframe containing sensor data.
            feature_columns (list): List of column names for features.
            window_size (int): The number of timesteps in each sliding window.
            step_size (int): The step size to slide the window.
            max_gap (int): Maximum allowed gap between consecutive indices for grouping.
        """
        self.features = dataframe[feature_columns].values
        self.indices = dataframe.index.values
        self.window_size = window_size
        self.step_size = step_size
        self.max_gap = max_gap
        self.stride = stride

        # Identify groups based on index gaps
        self.groups = self._identify_groups()
        # print(self.groups[0:50])
        # print(len(self.groups[500]))
        self.valid_windows = self._generate_valid_windows()

    def _identify_groups(self):
        """
        Identify groups of rows based on the max_gap condition.
        """
        groups = []
        current_group = [0]  # Start with the first row
        for i in range(1, len(self.indices)):
            if self.indices[i] - self.indices[i - 1] > self.max_gap:
                groups.append(current_group)
                current_group = [i]
            else:
                current_group.append(i)
        groups.append(current_group)  # Add the last group
        return groups

    def _generate_valid_windows(self):
        """
        Generate valid sliding windows based on groups.
        """
        valid_windows = []
        for group in self.groups:
            
            for start in range(0, len(group) - self.window_size + 1, self.stride):
                valid_windows.append(group[start : start + self.window_size])

            if (len(group) - self.window_size)%self.stride != 0 and len(group)> self.window_size:
                # print(len(group[len(group) - self.window_size : len(group)]))
                valid_windows.append(group[len(group) - self.window_size : len(group)])

        # print(valid_windows[-1])
        return valid_windows

    def __len__(self):
        return len(self.valid_windows)

    def __getitem__(self, idx):
        """
        Retrieve a sliding window by index.
        """
        window_indices = self.valid_windows[idx]
        x = torch.tensor(self.features[window_indices], dtype=torch.float32)
        return x

feature_columns = df.columns
# feature_columns = ["charge"]
# feature_columns = df.columns.to_string()
window_size = 100
stride = 10
max_gap = 10
batch_size = 32
step_size = 1  # Step size for sliding window

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle = False)
print("Total df shape:", df.shape)
print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)


# Initialize a scaler
scaler = MinMaxScaler()

# Fit the scaler on the training data
scaler.fit(train_df[feature_columns])

# Transform the training and testing data
train_df[feature_columns] = scaler.transform(train_df[feature_columns])
test_df[feature_columns] = scaler.transform(test_df[feature_columns])


# Create Dataset and DataLoader
train_dataset = SlidingWindowDataset(train_df, feature_columns, window_size, step_size, max_gap, stride = stride)
test_dataset = SlidingWindowDataset(test_df, feature_columns, window_size, step_size, max_gap, stride = stride)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

print("train dataloader length: ", len(train_dataloader))
print("train dataloader dataset length: ", len(train_dataloader.dataset))

print("test dataloader length: ", len(test_dataloader))
print("test dataloader dataset length: ", len(test_dataloader.dataset))

# maxes = []
# # Example: Iterating through the dataloader
# for i, batch in enumerate(dataloader):
#     sliding_windows = batch
#     diff = sliding_windows[0,1:10,1] - sliding_windows[0,0:9,1]
#     maxes.append(diff.max().item())
#     # print(maxes)  # Replace with your training loop logi
#     # if i>100:
#     #     break
# # plt.hist(maxes)
# pd_maxes = pd.DataFrame(maxes)
# pd_maxes.value_counts()

In [ ]:
class DenseAutoencoder(nn.Module):

    def __init__(self, input_dim):
        super(DenseAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            )
                          
        self.decoder = nn.Sequential(
            nn.Linear(32,128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, input_dim)
            # nn.Sigmoid()
            )

    def forward(self, x):
        feature = self.encoder(x)
        reconstruction = self.decoder(feature)
        return reconstruction, feature

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, print_every=10):
    total_loss = 0
    total_samples = 0

    for _ , x in enumerate(tqdm(dataloader)):
        batch_size = x.shape[0]
        total_samples += batch_size
        
        x = x.flatten(start_dim=1)
        optimizer.zero_grad()
        x_pred, _ = model(x)
        
        loss = loss_fn(x_pred, x)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * batch_size

    average_loss = total_loss / total_samples
    print(f"Train Loss: {average_loss:.3f}")
    return average_loss

def test_loop(dataloader, model, loss_fn):
    total_loss = 0
    total_samples = 0

    with torch.no_grad():
        for _, x in enumerate(tqdm(dataloader)):
            batch_size = x.shape[0]
            total_samples += batch_size

            x = x.flatten(start_dim=1)
            x_pred, _ = model(x)
            loss = loss_fn(x_pred, x)

            total_loss += loss.item() * batch_size 

    average_loss = total_loss / total_samples
    print(f"Test Loss: {average_loss:.3f}")
    return average_loss

model = DenseAutoencoder(window_size * len(feature_columns))
epochs = 10
# optimizer = torch.optim.AdamW(model.parameters(), lr = 0.001)


## Training loop with noise injection to improve robustness

In [ ]:
import torch
import torch.nn as nn

class ResidualConvAutoencoder(nn.Module):
    def __init__(self, input_dim, seq_len):
        super(ResidualConvAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv1d(input_dim, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.MaxPool1d(2),  # Down-sampling
            nn.Conv1d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.MaxPool1d(2)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(32, 64, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.ConvTranspose1d(64, 128, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, input_dim, kernel_size=3, stride=1, padding=1)
        )
        
    def forward(self, x):
        """
        Forward pass with skip connections.
        """
        # Add a channel dimension (needed for Conv1d)
        x = x.permute(0, 2, 1)  # Change shape from (batch_size, seq_len, features) to (batch_size, features, seq_len)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        decoded = decoded.permute(0, 2, 1)  # Back to (batch_size, seq_len, features)
        return decoded, encoded

# Define the model
input_dim = 89  # Number of features
seq_len = 100   # Window size
model = ResidualConvAutoencoder(input_dim, seq_len)

# Print the model architecture
# print(model)


In [ ]:
# Training settings
epochs = 100
batch_size = 32
lr = 0.001

# Optimizer and Loss
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
loss_fn = nn.MSELoss()

# Train the model
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer)


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, noise_factor=0.1, print_every=10):
    model.train()
    total_loss = 0
    total_samples = 0

    for _, x in enumerate(tqdm(dataloader)):
        batch_size = x.shape[0]
        total_samples += batch_size
        # x = x.flatten(start_dim=1)
        # Add noise to the input
        noisy_x = x + noise_factor * torch.randn_like(x)
        noisy_x = noisy_x.clamp(0., 1.)  # Ensure values stay within [0, 1]

        optimizer.zero_grad()
        x_pred, _ = model(noisy_x)
        
        loss = loss_fn(x_pred, x)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * batch_size

    average_loss = total_loss / total_samples
    print(f"Train Loss: {average_loss:.3f}")
    return average_loss



def test_loop(dataloader, model, loss_fn):
    total_loss = 0
    total_samples = 0

    with torch.no_grad():
        for _, x in enumerate(tqdm(dataloader)):
            batch_size = x.shape[0]
            total_samples += batch_size

            # x = x.flatten(start_dim=1)
            x_pred, _ = model(x)
            loss = loss_fn(x_pred, x)

            total_loss += loss.item() * batch_size 

    average_loss = total_loss / total_samples
    print(f"Test Loss: {average_loss:.3f}")
    return average_loss

epochs = 100
batch_size = 32
lr = 0.001

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)
loss_fn = nn.MSELoss()


In [ ]:
%%time
train_loss_evol = []
test_loss_evol = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss_evol.append(train_loop(train_dataloader, model, loss_fn, optimizer))
    test_loss_evol.append(test_loop(test_dataloader, model, loss_fn))


In [ ]:
plt.plot((train_loss_evol), label = 'train')
plt.plot((test_loss_evol), label = 'test')
plt.legend()
plt.grid()
plt.ylabel('loss')
plt.xlabel('Epochs')
plt.title('Loss evolution')
